# FixSigMeta

## Imports

In [ ]:
from fastdebug.core import *

### print comments with idx

In [ ]:
a = {}
a.update({1: "b"})
a

{1: 'b'}

### collect cmt from dbprint and print

In [ ]:

class Fastdb():
    
    def __init__(self, 
                 src, # name of src code
                 env): # env = g, as g = globals()
        self.orisrc = src
        self.margin = 157
        self.outenv = env
        self.cmts = {}

        
    def dbprint(self, 
                # src, # the src func name, e.g., foo
                dbcode, # the srclines under investigation, can be either string or int
                cmt:str, # comment
                *codes, # a list of dbcodes
                expand:int=2): # span 2 lines of srcode up and down from the srcline investigated
        "Insert dbcodes under srclines under investigation, and create a new dbsrc function to replace the official one"

        src = self.orisrc
        if type(dbcode) == int: self.cmts.update({dbcode: cmt})

        printsrc(src, dbcode, cmt, expand)

        dbsrc = ""
        indent = 4
        onedbprint = False

        lst = inspect.getsource(src).split('\n')
        if not bool(lst[-1]): lst = lst[:-1]

        srclines = ""
        if type(dbcode) == int:
            srclines = lst[dbcode]
        else:
            srclines = dbcode

        for idx, l in zip(range(len(lst)), lst):
            if bool(l.strip()) and l.strip() in srclines:

                numindent = len(l) - len(l.strip())
                dbcodes = "dbprintinsert("
                count = 1
                for c in codes:
                    if count == len(codes):
                        dbcodes = dbcodes + '"' + c + '"' + "," + "env=g" + ")"
                    else:
                        dbcodes = dbcodes + '"' + c + '"' + ","
                    count = count + 1

                if onedbprint == False:
                    dbsrc = dbsrc + " "*numindent + "g = locals()" + '\n'
                    dbsrc = dbsrc + " "*numindent + dbcodes + '\n'
                    dbsrc = dbsrc + l + '\n'
                    onedbprint = True
                else:
                    dbsrc = dbsrc + l + '\n'

            elif bool(l.strip()) and idx + 1 == len(lst):
                dbsrc = dbsrc + l

            elif bool(l.strip()): # make sure pure indentation + \n is ignored
                dbsrc = dbsrc + l + '\n'

        exec(dbsrc, globals().update(self.outenv)) # make sure b can access lst from above

        self.outenv.update(locals())

        return locals()[self.orisrc.__name__]
    
    
    def printsrcwithidx(self, 
                        maxlines:int=33, # maximum num of lines per page
                        part:int=0): # if the src is more than 33 lines, then divide the src by 33 into a few parts
        totallen = 157
        lenidx = 5
        lspace = 10
        lstsrc = inspect.getsource(self.orisrc).split('\n')
        numparts = len(lstsrc) // 33 + 1 if len(lstsrc) % 33 != 0 else len(lstsrc) // 33
        # cmts = {5:"this is me", 111:"this is me", 14:"this is you this is you this is you this is you this is you this is you this is you this is you "}
        cmts = self.cmts
        if part == 0: 
            for idx, l in zip(range(len(lstsrc)), lstsrc):
                lenl = len(l)
                
                if not bool(l.strip()):
                    print(l + " "*(totallen-lenl-lenidx) + "(" + str(idx) + ")")
                    
                elif lenl + lspace >= 100:
                    if bool(cmts):
                        cmtidx = [cmt[0] for cmt in list(cmts.items())]
                        if idx in cmtidx:
                            print(l + " # " + cmts[idx] + " "*(totallen-lenl-lenidx-len(cmts[idx])-3) + "(" + str(idx) + ")")
                        else:
                            print(l + " "*(totallen-lenl-lenidx) + "(" + str(idx) + ")")
                    else: 
                        print(l + " "*(totallen-lenl-lenidx) + "(" + str(idx) + ")")
                        
                else:


                    if bool(cmts):
                        cmtidx = [cmt[0] for cmt in list(cmts.items())]
                        if idx in cmtidx:
                            print('{:<100}'.format(l + "="*(100-lenl-lspace) + f"({idx})" + " # " + cmts[idx]))
                        else:
                            print('{:<100}'.format(l + "="*(100-lenl-lspace) + f"({idx})"))                                                      

                    else:
                        print('{:<100}'.format(l + "="*(100-lenl-lspace) + f"({idx})"))                 
        
        for p in range(numparts):
            for idx, l in zip(range(len(lstsrc)), lstsrc):

                if (maxlines*p <= idx < maxlines*(p+1) and p+1 == part):
                    lenl = len(l)
                    if not bool(l.strip()):
                        print(l + " "*(totallen-lenl-lenidx) + "(" + str(idx) + ")")
                    elif lenl + lspace >= 100:
                        if bool(cmts):
                            cmtidx = [cmt[0] for cmt in list(cmts.items())]
                            if idx in cmtidx:
                                print(l + " # " + cmts[idx] + " "*(totallen-lenl-lenidx-len(cmts[idx])-3) + "(" + str(idx) + ")")
                            else:
                                print(l + " "*(totallen-lenl-lenidx) + "(" + str(idx) + ")")
                        else: 
                            print(l + " "*(totallen-lenl-lenidx) + "(" + str(idx) + ")")


                    else:

                        if bool(cmts):
                            cmtidx = [cmt[0] for cmt in list(cmts.items())]
                            if idx in cmtidx:
                                print('{:<100}'.format(l + "="*(100-lenl-lspace) + f"({idx})" + " # " + cmts[idx]))
                            else:
                                print('{:<100}'.format(l + "="*(100-lenl-lspace) + f"({idx})"))                                                          
                            
                        else:
                            print('{:<100}'.format(l + "="*(100-lenl-lspace) + f"({idx})"))                      

                if (idx == maxlines*(p+1) or idx == len(lstsrc) - 1) and p+1 == part:
                    print('{:>157}'.format(f"part No.{p+1} out of {numparts} parts"))
                    return

In [ ]:
{1:'ab', 2:'cd'}[2]

'cd'

In [ ]:
list({1:'ab', 2:'cd'}.items())

[(1, 'ab'), (2, 'cd')]

In [ ]:
import inspect
from inspect import *
from inspect import _signature_from_callable
from inspect import _signature_is_functionlike, _signature_is_builtin, _signature_get_user_defined_method, _signature_from_function, _signature_bound_method

In [ ]:
from inspect import _signature_from_callable

In [ ]:
_signature_from_callableNew = _signature_from_callable

In [ ]:
def _signature_from_callableOld(obj, *,
                             follow_wrapper_chains=True,
                             skip_bound_arg=True,
                             sigcls):

    """Private helper function to get signature for arbitrary
    callable objects.
    """

    if not callable(obj):
        raise TypeError('{!r} is not a callable object'.format(obj))

    if isinstance(obj, types.MethodType):
        # In this case we skip the first parameter of the underlying
        # function (usually `self` or `cls`).
        sig = _signature_from_callable(
            obj.__func__,
            follow_wrapper_chains=follow_wrapper_chains,
            skip_bound_arg=skip_bound_arg,
            sigcls=sigcls)

        if skip_bound_arg:
            return _signature_bound_method(sig)
        else:
            return sig

    # Was this function wrapped by a decorator?
    if follow_wrapper_chains:
        obj = unwrap(obj, stop=(lambda f: hasattr(f, "__signature__")))
        if isinstance(obj, types.MethodType):
            # If the unwrapped object is a *method*, we might want to
            # skip its first parameter (self).
            # See test_signature_wrapped_bound_method for details.
            return _signature_from_callable(
                obj,
                follow_wrapper_chains=follow_wrapper_chains,
                skip_bound_arg=skip_bound_arg,
                sigcls=sigcls)

    try:
        sig = obj.__signature__
    except AttributeError:
        pass
    else:
        if sig is not None:
            if not isinstance(sig, Signature):
                raise TypeError(
                    'unexpected object {!r} in __signature__ '
                    'attribute'.format(sig))
            return sig

    try:
        partialmethod = obj._partialmethod
    except AttributeError:
        pass
    else:
        if isinstance(partialmethod, functools.partialmethod):
            # Unbound partialmethod (see functools.partialmethod)
            # This means, that we need to calculate the signature
            # as if it's a regular partial object, but taking into
            # account that the first positional argument
            # (usually `self`, or `cls`) will not be passed
            # automatically (as for boundmethods)

            wrapped_sig = _signature_from_callable(
                partialmethod.func,
                follow_wrapper_chains=follow_wrapper_chains,
                skip_bound_arg=skip_bound_arg,
                sigcls=sigcls)

            sig = _signature_get_partial(wrapped_sig, partialmethod, (None,))
            first_wrapped_param = tuple(wrapped_sig.parameters.values())[0]
            if first_wrapped_param.kind is Parameter.VAR_POSITIONAL:
                # First argument of the wrapped callable is `*args`, as in
                # `partialmethod(lambda *args)`.
                return sig
            else:
                sig_params = tuple(sig.parameters.values())
                assert (not sig_params or
                        first_wrapped_param is not sig_params[0])
                new_params = (first_wrapped_param,) + sig_params
                return sig.replace(parameters=new_params)

    if isfunction(obj) or _signature_is_functionlike(obj):
        # If it's a pure Python function, or an object that is duck type
        # of a Python function (Cython functions, for instance), then:
        return _signature_from_function(sigcls, obj)

    if _signature_is_builtin(obj):
        return _signature_from_builtin(sigcls, obj,
                                       skip_bound_arg=skip_bound_arg)

    if isinstance(obj, functools.partial):
        wrapped_sig = _signature_from_callable(
            obj.func,
            follow_wrapper_chains=follow_wrapper_chains,
            skip_bound_arg=skip_bound_arg,
            sigcls=sigcls)
        return _signature_get_partial(wrapped_sig, obj)

    sig = None
    if isinstance(obj, type):
        # obj is a class or a metaclass

        # First, let's see if it has an overloaded __call__ defined
        # in its metaclass
        call = _signature_get_user_defined_method(type(obj), '__call__')
        if call is not None:
            sig = _signature_from_callable(
                call,
                follow_wrapper_chains=follow_wrapper_chains,
                skip_bound_arg=skip_bound_arg,
                sigcls=sigcls)
        else:
            # Now we check if the 'obj' class has a '__new__' method
            new = _signature_get_user_defined_method(obj, '__new__')
            if new is not None:
                sig = _signature_from_callable(
                    new,
                    follow_wrapper_chains=follow_wrapper_chains,
                    skip_bound_arg=skip_bound_arg,
                    sigcls=sigcls)
            else:
                # Finally, we should have at least __init__ implemented
                init = _signature_get_user_defined_method(obj, '__init__')
                if init is not None:
                    sig = _signature_from_callable(
                        init,
                        follow_wrapper_chains=follow_wrapper_chains,
                        skip_bound_arg=skip_bound_arg,
                        sigcls=sigcls)

        if sig is None:
            # At this point we know, that `obj` is a class, with no user-
            # defined '__init__', '__new__', or class-level '__call__'

            for base in obj.__mro__[:-1]:
                # Since '__text_signature__' is implemented as a
                # descriptor that extracts text signature from the
                # class docstring, if 'obj' is derived from a builtin
                # class, its own '__text_signature__' may be 'None'.
                # Therefore, we go through the MRO (except the last
                # class in there, which is 'object') to find the first
                # class with non-empty text signature.
                try:
                    text_sig = base.__text_signature__
                except AttributeError:
                    pass
                else:
                    if text_sig:
                        # If 'obj' class has a __text_signature__ attribute:
                        # return a signature based on it
                        return _signature_fromstr(sigcls, obj, text_sig)

            # No '__text_signature__' was found for the 'obj' class.
            # Last option is to check if its '__init__' is
            # object.__init__ or type.__init__.
            if type not in obj.__mro__:
                # We have a class (not metaclass), but no user-defined
                # __init__ or __new__ for it
                if (obj.__init__ is object.__init__ and
                    obj.__new__ is object.__new__):
                    # Return a signature of 'object' builtin.
                    return sigcls.from_callable(object)
                else:
                    raise ValueError(
                        'no signature found for builtin type {!r}'.format(obj))

    elif not isinstance(obj, _NonUserDefinedCallables):
        # An object with __call__
        # We also check that the 'obj' is not an instance of
        # _WrapperDescriptor or _MethodWrapper to avoid
        # infinite recursion (and even potential segfault)
        call = _signature_get_user_defined_method(type(obj), '__call__')
        if call is not None:
            try:
                sig = _signature_from_callable(
                    call,
                    follow_wrapper_chains=follow_wrapper_chains,
                    skip_bound_arg=skip_bound_arg,
                    sigcls=sigcls)
            except ValueError as ex:
                msg = 'no signature found for {!r}'.format(obj)
                raise ValueError(msg) from ex

    if sig is not None:
        # For classes and objects we skip the first parameter of their
        # __call__, __new__, or __init__ methods
        if skip_bound_arg:
            return _signature_bound_method(sig)
        else:
            return sig

    if isinstance(obj, types.BuiltinFunctionType):
        # Raise a nicer error message for builtins
        msg = 'no signature found for builtin function {!r}'.format(obj)
        raise ValueError(msg)

    raise ValueError('callable {!r} is not supported by signature'.format(obj))

## When or why to use `FixSigMeta`?

When we want a class to use signature of `__init__` to be its class signature. 

In [ ]:
import inspect

In [ ]:
class Foo:
    def __init__(self, a, b, c): pass
    
inspect.signature(Foo)

<Signature (a, b, c)>

## How Foo borrow sig from `__init__`

In [ ]:
g = globals()
sig = Fastdb(_signature_from_callable, g)

In [ ]:
sig.printsrcwithidx(part=1) # the conditions and comments are well informed

def _signature_from_callable(obj, *,======================================================(0)       
                             follow_wrapper_chains=True,==================================(1)       
                             skip_bound_arg=True,=========================================(2)       
                             sigcls):=====================================================(3)       
                                                                                                                                                        (4)
    """Private helper function to get signature for arbitrary=============================(5)       
    callable objects.=====================================================================(6)       
    """===================================================================================(7)       
                                                                                                                                         

In [ ]:
sig.printsrcwithidx(part=4) # the conditions and comments are well informed

        else:=============================================================================(99)      
            factory_method = None=========================================================(100)     
            new = _signature_get_user_defined_method(obj, '__new__')======================(101)     
            init = _signature_get_user_defined_method(obj, '__init__')====================(102)     
            # Now we check if the 'obj' class has an own '__new__' method=================(103)     
            if '__new__' in obj.__dict__:=================================================(104)     
                factory_method = new======================================================(105)     
            # or an own '__init__' method=================================================(106)     
            elif '__init__' in obj.__dict__:==============================================(107)     
                factory_method = init=====================================================(

In [ ]:
sig.dbprint(15, "just read the first line and comments", "", expand=1)

    if not callable(obj):                                                                                                                               (14)
        raise TypeError('{!r} is not a callable object'.format(obj))====================================================================================(15)
                                                                                                                        just read the first line and comments
                                                                                                                                                        (16)


<function __main__._signature_from_callable(obj, *, follow_wrapper_chains=True, skip_bound_arg=True, sigcls)>

In [ ]:
sig.printsrcwithidx(part=1) # the conditions and comments are well informed

def _signature_from_callable(obj, *,======================================================(0)       
                             follow_wrapper_chains=True,==================================(1)       
                             skip_bound_arg=True,=========================================(2)       
                             sigcls):=====================================================(3)       
                                                                                                                                                        (4)
    """Private helper function to get signature for arbitrary=============================(5)       
    callable objects.=====================================================================(6)       
    """===================================================================================(7)       
                                                                                                                                         

In [ ]:
sig.printsrcwithidx(part=3) # the conditions and comments are well informed

                # First argument of the wrapped callable is `*args`, as in================(66)      
                # `partialmethod(lambda *args)`.==========================================(67)      
                return sig================================================================(68)      
            else:=========================================================================(69)      
                sig_params = tuple(sig.parameters.values())===============================(70)      
                assert (not sig_params or=================================================(71)      
                        first_wrapped_param is not sig_params[0])=========================(72)      
                new_params = (first_wrapped_param,) + sig_params==========================(73)      
                return sig.replace(parameters=new_params)=================================(74)      
                                                                                           

In [ ]:
sig.printsrcwithidx??

Signature: sig.printsrcwithidx(maxlines: int = 33, part: int = 0)
Docstring: <no docstring>
Source:   
    def printsrcwithidx(self, 
                        maxlines:int=33, # maximum num of lines per page
                        part:int=0): # if the src is more than 33 lines, then divide the src by 33 into a few parts
        totallen = 157
        lenidx = 5
        lspace = 10
        lstsrc = inspect.getsource(self.orisrc).split('\n')
        numparts = len(lstsrc) // 33 + 1 if len(lstsrc) % 33 != 0 else len(lstsrc) // 33
        # cmts = {5:"this is me", 111:"this is me", 14:"this is you this is you this is you this is you this is you this is you this is you this is you "}
        cmts = self.cmts
        if part == 0: 
            for idx, l in zip(range(len(lstsrc)), lstsrc):
                lenl = len(l)
                
                if not bool(l.strip()):
                    print(l + " "*(totallen-lenl-lenidx) + "(" + str(idx) + ")")
                    
               

In [ ]:
# def printsrcwithidx(self, 
#                     maxlines:int=33, # maximum num of lines per page
#                     part:int=0): # if the src is more than 33 lines, then divide the src by 33 into a few parts
#     totallen = 157
#     lenidx = 5
#     lstsrc = inspect.getsource(self.orisrc).split('\n')
#     numparts = len(lstsrc) // 33 + 1 if len(lstsrc) % 33 != 0 else len(lstsrc) // 33

#     if part == 0: 
#         for idx, l in zip(range(len(lstsrc)), lstsrc):
#             lenl = len(l)
#             print(l + " "*(totallen-lenl-lenidx) + "(" + str(idx) + ")")


#     for p in range(numparts):
#         for idx, l in zip(range(len(lstsrc)), lstsrc):

#             if (maxlines*p <= idx < maxlines*(p+1) and p+1 == part):
#                 lenl = len(l)
#                 # print(l + " "*(totallen-lenl-lenidx) + "(" + str(idx) + ")")
#                 print(l + " "*lenidx + "(" + str(idx) + ")")

#             if (idx == maxlines*(p+1) or idx == len(lstsrc) - 1) and p+1 == part:
#                 print('{:>157}'.format(f"part No.{p+1} out of {numparts} parts"))
#                 # return

Many things can go wrong to prevent a class to use the signature from `__init__`. 

FixSigMeta is a metaclass, which helps us to get our classes' signature right.

Then what types of the signature problems can FixSigMeta fix?

1. when your class Foo inherits from class Base, if Base defines its `__new__`, then Foo can't get signature from `__init__`. (True for python 3.7 see [demos](https://www.kaggle.com/code/danielliao/notebook3edc928f49?scriptVersionId=104385507&cellId=1), no more for 3.9+)

In [ ]:
class Base: # pass
    def __new__(self, **args): pass  # defines a __new__ 

class Foo(Base):
    def __init__(self, d, e, f): pass
    
inspect.signature(Foo) # no more problem for python 3.9+, 

<Signature (d, e, f)>

In [ ]:
from IPython.display import IFrame

IFrame(src="https://www.kaggle.com/embed/danielliao/notebook3edc928f49?cellIds=2&kernelSessionId=104407182", width = "1200", height="300", \
       style="margin: 0 auto; width: 100%; max-width: 950px;", frameborder="0", scrolling="auto", title="notebook3edc928f49")

In [ ]:
inspect._signature_from_callable = _signature_from_callableOld
inspect.signature(Foo) # it is a problem for python 3.7, 

<Signature (**args)>

In [ ]:
inspect._signature_from_callable = _signature_from_callableNew

Solution to 1: By also inheriting from the metaclass FixSigMeta can solve the signature problem for Foo (for python 3.7)

In [ ]:
from fastcore.meta import FixSigMeta, test_sig

In [ ]:
class Base: # pass
    def __new__(self, **args): pass  # defines a __new__ 

class Foo(Base, metaclass=FixSigMeta):
    def __init__(self, d, e, f): pass
    
test_sig(Foo, '(d, e, f)')
inspect.signature(Foo)

<Signature (d, e, f)>

In [ ]:
IFrame(src="https://www.kaggle.com/embed/danielliao/notebook3edc928f49?cellIds=4&kernelSessionId=104407182", width = "1200", height="300", \
       style="margin: 0 auto; width: 100%; max-width: 950px;", frameborder="0", scrolling="auto", title="notebook3edc928f49")

In [ ]:
inspect._signature_from_callable = _signature_from_callableOld
inspect.signature(Foo) # it is a problem for python 3.7, 

<Signature (d, e, f)>

In [ ]:
inspect._signature_from_callable = _signature_from_callableNew

2. when your Foo has a metaclass BaseMeta, if BaseMeta need to define its `__call__`, then Foo can't get signature from `__init__`.

In [ ]:
class BaseMeta(type): 
    # using __new__ from type
    def __call__(cls, *args, **kwargs): pass
class Foo(metaclass=BaseMeta): 
    def __init__(self, d, e, f): pass

test_sig(Foo, '(*args, **kwargs)')

In [ ]:
class BaseMeta(type): 
    def __new__(cls, name, bases, dict):
        return super().__new__(cls, name, bases, dict) # using __new__ from type
    def __call__(cls, *args, **kwargs): pass
class Foo(metaclass=BaseMeta): 
    def __init__(self, d, e, f): pass

test_sig(Foo, '(*args, **kwargs)')

Solution to problem 2: you need to inherit from FixSigMeta instead of type when constructing the metaclass to preserve the signature in `__init__`. Be careful not to override `__new__` when doing this:

In [ ]:
class BaseMeta(FixSigMeta): 
    # using __new__ of  FixSigMeta instead of type
    def __call__(cls, *args, **kwargs): pass

class Foo(metaclass=BaseMeta): # Base
    def __init__(self, d, e, f): pass

test_sig(Foo, '(d, e, f)')

In [ ]:
class BaseMeta(FixSigMeta): 
    def __new__(cls, name, bases, dict): # not really overriding __new__, but still using FixSigMeta.__new__
        return super().__new__(cls, name, bases, dict)
    def __call__(cls, *args, **kwargs): pass

class Foo(metaclass=BaseMeta): # Base
    def __init__(self, d, e, f): pass

test_sig(Foo, '(d, e, f)')

Note: if Base also defines `__new__`, then FixSigMeta can't help. 

In [ ]:
from fastcore.meta import test_eq

In [ ]:
class BaseMeta(FixSigMeta): 
    # __new__ comes from FixSigMeta
    def __new__(cls, *args, **kwargs): pass # actually overriding __new__, not using FixSigMeta.__new__ anymore
    def __call__(cls, *args, **kwargs): pass

class Foo(metaclass=BaseMeta): # Base
    def __init__(self, d, e, f): pass

test_eq(type(Foo), type(None))

Note: if Base also defines `__init__`, then FixSigMeta can still help. 

In [ ]:
class BaseMeta(FixSigMeta): 
    # __new__ comes from FixSigMeta
    def __init__(cls, *args, **kwargs): pass # overriding __init__ of FixSigMeta, is fine
    def __call__(cls, *args, **kwargs): pass

class Foo(metaclass=BaseMeta): # Base
    def __init__(self, d, e, f): pass

test_sig(Foo, '(d, e, f)')